In [ ]:
from numba import cuda
import numpy as np

def verify(res, b, a):
    return not False in [i for i in res==np.dot(b,a)]

def matrix_vector_multiply(matrix, vector, shape):
    result=np.empty_like(vector)
    for i in range(len(vector)):
        result[i]=0
        for j in range(shape[0]):
            result[i]+=matrix[i][j]*vector[j]
            #print(result)
    return result

@cuda.jit
def matrix_vector_multiply_parallel(matrix, vector, shape, out):
    th_idx = cuda.grid(1)
    for i in range(shape[0]):
        cuda.atomic.add(out, i, matrix[i][th_idx] * vector[th_idx])

@cuda.jit
def matrix_vector_multiply_parallel_stride(matrix, vector, shape, stride, out):
    th_idx = cuda.grid(1)
    for i in range(shape[0]):
      for j in range(stride*th_idx, stride*th_idx+stride):
        cuda.atomic.add(out, i, matrix[i][j] * vector[j])

In [ ]:
import numpy as np
import random

def gen_int_matr(shape, start, end):
    b=np.empty(shape, dtype=np.int16)
    for i in range(shape[1]):
        for j in range(shape[0]):
            b[i][j]=random.randint(start, end)
    return b

def gen_float_matr(shape, start, end):
    b=np.empty(shape, dtype=np.float16)
    for i in range(shape[1]):
        for j in range(shape[0]):
            b[i][j]=random.uniform(start, end)
    return b

In [ ]:
shape=(16, 16)

#vector
a=np.arange(0, shape[0]).astype(np.int32)

In [ ]:
#testing cpu version
results=[]
for _ in range(0, 10000):
    b=gen_int_matr(shape, 0, 500)
    res = matrix_vector_multiply(b, a, shape)
    results.append(verify(res, b, a))
    if results[-1]==False:
        print("b:\n", b)
        print("a:\n", a)
        print("res:\n", res)
        print("dot: \n", np.dot(b, a))
print("testing results: ", results.count(True), " out of ", len(results))

testing results:  10000  out of  10000


In [ ]:
#testing gpu version
gpu_config = (4, 4)
results = []

for _ in range(0, 10000):
  b = gen_int_matr(shape, 0, 500)
  result = np.zeros(shape[0]).astype(np.int32)
  b_gpu = cuda.to_device(b)
  a_gpu = cuda.to_device(a)
  matrix_vector_multiply_parallel[gpu_config](b_gpu, a_gpu, shape, result)
  results.append(verify(result, b, a))
print("testing results: ", results.count(True), " out of ", len(results))

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


testing results:  10000  out of  10000


In [ ]:
# big test
shape = (4096, 4096)

a = np.arange(0, shape[0]).astype(np.int32)
b = gen_int_matr(shape, 0, 500)

In [ ]:
a_gpu = cuda.to_device(a)
b_gpu = cuda.to_device(b)
gpu_config = (64, 64)
out = np.zeros(shape[0]).astype(np.int32)

In [ ]:
%timeit matrix_vector_multiply_parallel[gpu_config](b_gpu, a_gpu, shape, out)

2.28 ms ± 42.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit matrix_vector_multiply(b, a, shape)

<ipython-input-18-8dd1e2bbc7e5>:12: RuntimeWarning: overflow encountered in scalar add
  result[i]+=matrix[i][j]*vector[j]


13.8 s ± 306 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
a_gpu = cuda.to_device(a)
b_gpu = cuda.to_device(b)
stride = 2
gpu_config = (64, 32)
out = np.zeros(shape[0]).astype(np.int32)

In [ ]:
%timeit matrix_vector_multiply_parallel_stride[gpu_config](b_gpu, a_gpu, shape, stride, out)

3.24 ms ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
